In [3]:
from configs.config import Config
from src.datasets.course_dataset import CourseDataset

cfg = Config()
dataset = CourseDataset(cfg.data)

print(dataset[10])

{'control_number': 'ccc000392714', 'title': 'WE: Issues-Fashion Design', 'description': '<span style="font-face:ArialMT;font-size:x-small;"><span style="font-face:ArialMT;font-size:x-small;"><p align="LEFT">This course is a seminar related to work experience. Included is discussion of work experience objectives, career goals,</p><p align="LEFT">employment search, communication skills and problem solving. Creation of resumes and cover letters will be covered and</p><p>completed.</p></span></span>', 'embedding': tensor([-0.0049, -0.0239, -0.0014,  ..., -0.0030, -0.0011, -0.0335])}


In [5]:
len(dataset)

132214